In [52]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

In [53]:
%run '/Volumes/GoogleDrive/My Drive/Estudos/IC/Henrique/Programs/RMlicelPython/profile_read_py.py'

# Reading multiple data (mean signal)

The idea here is to iterate over the data files in a month directory.

In [152]:
import os

month = '/Users/costaleirbag/Downloads/3'

# Primeiro testo um arquivo pra saber o tamanho do array gerado, e ai gerar o dataframe
# ja certo.

days_folder = os.listdir(month)
days_folder.sort()
names = []

#os.mkdir(f'Plots/Month{os.path.basename(month)}')

for day in days_folder:
    directory = month + f'/{day}'
    files = os.listdir(directory)
    files.sort()
    
    n_files = len(files)
    print(f'Number of files at day {day}: {n_files}')
    
    data    = np.zeros((array_size, n_files))
    date    = []

    cont    = 0
    errors  = []
    nodata  = []
    
    for filename in files:
        #print(f'Arquivo: {cont+1}', end='\r')
        path = os.path.join(directory, filename)
        try:
            head, phy, raw = profile_read(path)
            date_str = head['datei'] + ' ' + head['houri']
            date.append(datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S"))
            data[:, cont] = phy[:, 0]
        except ValueError as e:
            nodata.append(str(cont))
            #print(f'Value error happened at file {cont} in day {day}.')
            #print(f"{e}, {e.__class__}")
            data[:, cont] = np.nan
            date_str = head['datei'] + ' ' + head['houri']
            date.append(datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S"))
        except OverflowError as e:
            errors.append(str(cont))
            #print(f"Overflow error happened at file {cont}.")
            #print(f"{e}, {e.__class__}")
            data[:, cont] = np.nan
            date_str = head['datei'] + ' ' + head['houri']
            date.append(datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S"))
        cont += 1
        #print(data)

    #date = np.array(date)

    errors_str = ', '.join(errors)
    nodata_str = ', '.join(nodata)
    print(f'\n  All files with overflow: {errors_str}.\n')
    if len(nodata) != 0:
        print(f'\n  All files with no data: {nodata_str}.\n')
        
    
    
    bg_vec = np.mean(data[-500:], axis=0)
    data_nobg = data - bg_vec
    
    data_pr2 = pd.DataFrame((data_nobg.T * alt * alt).T, columns=date)
    data_pr2['Altitude'] = alt
    data_pr2.set_index('Altitude', inplace=True)


    # Tirando medias de 5 em 5 min
    data_pr2.columns = pd.to_datetime(data_pr2.columns)
    data_pr2 = data_pr2.resample('5T', axis=1).mean()

    fig, ax = plt.subplots(figsize = (12,9))

    psm = ax.pcolormesh(data_pr2.columns, alt*1e-3, (data_pr2), vmin = 0, vmax=2.6e7, cmap='jet')

    cbar = fig.colorbar(psm, ax=ax, label = "Range Corrected Signal [a. u.]")

    ax.set_xlabel("Date", fontsize=20)

    ax.set_ylabel("Altitude (km)", fontsize=20)

    ax.set_ylim(0.75, 5)

    cbar.ax.tick_params(labelsize=15)

    ax.figure.axes[-1].yaxis.label.set_size(20)

    date_form = DateFormatter("%d/%m/%Y \n%H:%M:%S")
    ax.xaxis.set_major_formatter(date_form)
    
    plt.savefig(f'Plots/Month{os.path.basename(month)}/Day{day}.png', dpi=100)
    plt.close()

Number of files at day 15: 274

  All files with overflow: 6, 18, 27, 168, 186, 195.

Number of files at day 16: 654

  All files with overflow: 14.

Number of files at day 20: 439

  All files with overflow: 107.

Number of files at day 21: 770

  All files with overflow: 208, 258.

Number of files at day 22: 1197

  All files with overflow: 28, 65, 315, 349, 553, 570.

Number of files at day 23: 715

  All files with overflow: 698.

Number of files at day 24: 1105

  All files with overflow: 169, 380, 782, 817.

Number of files at day 25: 1239

  All files with overflow: 21, 57, 278, 402, 425, 671, 847, 1207, 1208.

Number of files at day 26: 1239

  All files with overflow: 396, 409, 1060, 1074.

Number of files at day 27: 1239

  All files with overflow: 205, 667, 880, 969, 998, 1026, 1045, 1194.

Number of files at day 28: 650

  All files with overflow: 20, 36, 165, 282.

Number of files at day 29: 188

  All files with overflow: .

Number of files at day 30: 1239

  All files wi